# 1. 데이터 로드

In [1]:
import os
import sys
import time
import random
import datetime
import requests
import pandas as pd
import numpy as np
import hashlib, hmac, base64
from itertools import combinations, permutations
from dtw import *
import json
import urllib.request
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.api as sm
import pickle
from pytz import timezone
from difflib import SequenceMatcher

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from collections import defaultdict
from pytrends.request import TrendReq


Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



# 2. API설정

In [2]:
from api_set import APIClient

# API 설정
from utils import get_secret
BASE_URL = get_secret("BASE_URL")
CUSTOMER_ID = get_secret("CUSTOMER_ID")
API_KEY = get_secret("API_KEY")
SECRET_KEY = get_secret("SECRET_KEY")
URI = get_secret("URI")
METHOD = get_secret("METHOD")
# API 클라이언트 인스턴스 생성
api_client = APIClient(BASE_URL, CUSTOMER_ID, API_KEY, SECRET_KEY,URI,METHOD)


# 3. 연관검색어 수집

In [3]:
# 키 로드
from utils import load_keywords 
keywords_data = load_keywords('main_keyword.json')

from utils import get_today_date
# 오늘의 날짜 가져오기
formatted_today, day = get_today_date()


# 결과 저장 폴더 생성
from utils import make_directory

make_directory('./data')
make_directory('./data/rl_srch')
make_directory(f'./data/rl_srch/{day}')  # 키워드별 연관검색어 리스트 저장

In [4]:

# 검색어 리스트와 결과 저장 경로 설정
srch_keyword = ['keyword_final']  
save_path = './data/rl_srch/'  
print(api_client.base_url)

https://api.searchad.naver.com


In [5]:
srch_keyword

['keyword_final']

In [6]:
import os
import csv
import datetime
import asyncio
import pandas as pd

# 필요한 경우 비동기를 위한 nest_asyncio 적용
import nest_asyncio
nest_asyncio.apply()

from collect_keywords import collect_keywords

async def main(srch_keyword, day):
    # 오늘 날짜로 폴더 경로 생성
    folder_path = './data/rl_srch/' + datetime.datetime.now().strftime('%y%m%d')
    file_path = f"{folder_path}/collected_keywords.csv"
    
    # 폴더가 존재하는지 확인
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    # 파일이 존재하는지 확인
    if os.path.isfile(file_path):
        # 파일이 존재하면, 데이터를 읽어옵니다.
        collected_keywords_data = pd.read_csv(file_path)
    else:
        # 파일이 없으면, collect_keywords 함수를 호출해서 데이터를 수집합니다.
        collected_keywords_data = await collect_keywords(srch_keyword, day)
        # 결과를 CSV로 저장
        collected_keywords_data.to_csv(file_path, index=False)
    
    return collected_keywords_data
collected_keywords_data=asyncio.run(main(srch_keyword, day))

In [7]:
print(collected_keywords_data)

         연관키워드  월간검색수_합계 검색어
0           주식  528500.0  주식
1          미주부     370.0  주식
2        김현준대표     940.0  주식
3         퀀트투자    6650.0  주식
4         주식투자   12310.0  주식
...        ...       ...  ..
30599   수원유언공증      18.0  증여
30600   법인주소변경    1940.0  증여
30601   인터넷법무사     460.0  증여
30602    환지예정지     330.0  증여
30603  지정유언집행자      80.0  증여

[30604 rows x 3 columns]


In [8]:
sorted_df = collected_keywords_data.groupby('검색어').apply(lambda x: x.sort_values('월간검색수_합계', ascending=False)).reset_index(drop=True)

# 각 '검색어'별로 분리된 DataFrame을 리스트에 저장
grouped = sorted_df.groupby('검색어')
df_list = [group for _, group in grouped]
print(len(df_list))

45


C:\Users\chohy\AppData\Local\Temp\ipykernel_32508\3376418344.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sorted_df = collected_keywords_data.groupby('검색어').apply(lambda x: x.sort_values('월간검색수_합계', ascending=False)).reset_index(drop=True)


In [9]:
from utils import merge_and_mark_duplicates_limited
collected_keywords_data = merge_and_mark_duplicates_limited(df_list)


collected_keywords_data



,연관키워드,월간검색수_합계,검색어,중복검색어
0,달러환율,2050600.0,CMA금리비교,"CMA금리비교,WTI,달러환율,미국금리,미국주식,주가지수,주식"
1,적금계산기,644900.0,CMA금리비교,"CMA금리비교,CMA통장"
2,다우지수,561700.0,CMA금리비교,"CMA금리비교,CMA통장,ETF,WTI,급등주,달러환율,미국금리,미국주식,테마주"
3,적금,470400.0,CMA금리비교,"CMA금리비교,CMA통장,금리,급등주,돈버는법,미국금리,배당주,재테크,주가지수,주식"
4,코스닥,450600.0,CMA금리비교,"CMA금리비교,CMA통장,ETF,WTI,개인연금,달러환율,배당주,신규상장,외국인순매..."
...,...,...,...,...
1141,S&P500지수,33530.0,테마주,"테마주,퇴직연금"
1142,TQQQ,32650.0,테마주,테마주
1143,코스피야간선물,31270.0,테마주,테마주
1144,양자컴퓨터관련주,30430.0,테마주,테마주


In [10]:
import utils

def add_client_info(collected_keywords_data, start_id_index=1):
    clients = utils.get_secret("clients")
    start_id_index = 1
    clients = utils.get_secret("clients")
    # ID와 PW 컬럼을 데이터프레임에 추가하는 로직
    total_rows = len(collected_keywords_data)
    ids = []
    pws = []

    for i in range(total_rows):
        # 현재 id 인덱스 계산 (start_id_index를 기준으로)
        current_id_index = ((i // 500) + start_id_index) % len(clients)
        current_id_key = f"id_{current_id_index}"
        
        # 현재 id와 pw 할당
        current_id = clients[current_id_key]['client_id']
        current_pw = clients[current_id_key]['client_secret']
        
        ids.append(current_id)
        pws.append(current_pw)

    # ID와 PW 컬럼 추가
    collected_keywords_data['id'] = ids
    collected_keywords_data['pw'] = pws

    return collected_keywords_data
collected_keywords_data= add_client_info(collected_keywords_data)
collected_keywords_data

,연관키워드,월간검색수_합계,검색어,중복검색어,id,pw
0,달러환율,2050600.0,CMA금리비교,"CMA금리비교,WTI,달러환율,미국금리,미국주식,주가지수,주식",UcxKdYQZjj2FhdkZjgRz,uy9jFsFAOr
1,적금계산기,644900.0,CMA금리비교,"CMA금리비교,CMA통장",UcxKdYQZjj2FhdkZjgRz,uy9jFsFAOr
2,다우지수,561700.0,CMA금리비교,"CMA금리비교,CMA통장,ETF,WTI,급등주,달러환율,미국금리,미국주식,테마주",UcxKdYQZjj2FhdkZjgRz,uy9jFsFAOr
3,적금,470400.0,CMA금리비교,"CMA금리비교,CMA통장,금리,급등주,돈버는법,미국금리,배당주,재테크,주가지수,주식",UcxKdYQZjj2FhdkZjgRz,uy9jFsFAOr
4,코스닥,450600.0,CMA금리비교,"CMA금리비교,CMA통장,ETF,WTI,개인연금,달러환율,배당주,신규상장,외국인순매...",UcxKdYQZjj2FhdkZjgRz,uy9jFsFAOr
...,...,...,...,...,...,...
1141,S&P500지수,33530.0,테마주,"테마주,퇴직연금",mZtQ5QquBFIrshEwfZBv,jAIZheOMh8
1142,TQQQ,32650.0,테마주,테마주,mZtQ5QquBFIrshEwfZBv,jAIZheOMh8
1143,코스피야간선물,31270.0,테마주,테마주,mZtQ5QquBFIrshEwfZBv,jAIZheOMh8
1144,양자컴퓨터관련주,30430.0,테마주,테마주,mZtQ5QquBFIrshEwfZBv,jAIZheOMh8


In [11]:
def groupped_df(name,collected_keywords_data):
    grouped = collected_keywords_data.groupby(name)
    df_list = [group for _, group in grouped]
    return df_list
df_list=groupped_df('id',collected_keywords_data)
print(len(df_list))

3


In [12]:
import asyncio
import trend  # 가정: trend 모듈에 trend_maincode 함수가 정의되어 있음

# 비동기 메인 함수 수정
async def trend_main(df, clients):
    # 파라미터 설정
    params = {
        "search_keywords": list(df['연관키워드']),
        "id": df['id'].iloc[0],
        "pw": df['pw'].iloc[0],
        "api_url": "https://openapi.naver.com/v1/datalab/search",
        "name": '연관검색어'
    }
    api_url = "https://openapi.naver.com/v1/datalab/search"
    
    # trend_maincode 함수 실행
    results = await trend.trend_maincode(params, clients, api_url)
    return results

async def run_all(df_list, clients):
    tasks = [trend_main(df, clients) for df in df_list]
    results = await asyncio.gather(*tasks)
    return results

clients = get_secret("clients")  # clients 정보를 로드
# 이벤트 루프 실행
results = asyncio.run(run_all(df_list, clients))

클라이언트 오류 발생: 429, 메시지='Too Many Requests, URL='https://openapi.naver.com/v1/datalab/search''
클라이언트 오류 발생: 429, 메시지='Too Many Requests, URL='https://openapi.naver.com/v1/datalab/search''
클라이언트 오류 발생: 429, 메시지='Too Many Requests, URL='https://openapi.naver.com/v1/datalab/search''
클라이언트 오류 발생: 429, 메시지='Too Many Requests, URL='https://openapi.naver.com/v1/datalab/search''
클라이언트 오류 발생: 429, 메시지='Too Many Requests, URL='https://openapi.naver.com/v1/datalab/search''
클라이언트 오류 발생: 429, 메시지='Too Many Requests, URL='https://openapi.naver.com/v1/datalab/search''
클라이언트 오류 발생: 429, 메시지='Too Many Requests, URL='https://openapi.naver.com/v1/datalab/search''
클라이언트 오류 발생: 429, 메시지='Too Many Requests, URL='https://openapi.naver.com/v1/datalab/search''
클라이언트 오류 발생: 429, 메시지='Too Many Requests, URL='https://openapi.naver.com/v1/datalab/search''
클라이언트 오류 발생: 429, 메시지='Too Many Requests, URL='https://openapi.naver.com/v1/datalab/search''
클라이언트 오류 발생: 429, 메시지='Too Many Requests, URL='https://openapi.naver.c

In [30]:
from select_keyword import select_keyword, rising_keyword_analysis, monthly_rule

start_time = time.time()
analysis_periods = ['daily', 'weekly', 'month']

formatted_today, today_date = utils.get_today_date()
month_rule=pd.DataFrame(columns=['tmp','graph','info','rising_month'])
rising_week=pd.DataFrame(columns=['tmp','graph','info'])
rising_month=pd.DataFrame(columns=['tmp','graph','info'])
select_daily=pd.DataFrame(columns=['tmp','graph','info'])
select_weekly=pd.DataFrame(columns=['tmp','graph','info'])
select_month=pd.DataFrame(columns=['tmp','graph','info'])
# 월별, 주별, 일별 키워드 분석 실행
for period in analysis_periods:
    for keyword_df_group in results:
        for keyword_df in keyword_df_group:
            monthly_tmp, monthly_graph, similarity_rt, rising_month = monthly_rule(keyword_df, today_date, period)
            if monthly_tmp is not None:
                print('type(month_tmp):',type(monthly_tmp))
                print('type(monthly_graph):',type(monthly_graph))
                print('type(similarity_rt):',type(similarity_rt))
                print('monthly_graph:',monthly_graph)
# # 주별, 월별 상승 키워드 분석 실행
# rising_analysis_periods = ['weekly', 'month']
# for period in rising_analysis_periods:
#     for keyword_df_group in results:
#         for keyword_df in keyword_df_group:
#             rising_tmp, rising_graph, rising_keywords = rising_keyword_analysis(keyword_df, today_date, period)


# # 일별, 주별, 월별 키워드 선택 실행
# for period in analysis_periods:
#     for keyword_df_group in results:
#         for keyword_df in keyword_df_group:
#             selected_tmp, selected_graph, selected_end = select_keyword(keyword_df, today_date, period)

end_time = time.time()
print(f"Analysis completed in {end_time - start_time} seconds.")

월별 규칙성 키워드 발견 : 소득공제
type(month_tmp): <class 'pandas.core.frame.DataFrame'>
type(monthly_graph): <class 'pandas.core.frame.DataFrame'>
type(similarity_rt): <class 'numpy.float64'>
monthly_graph:           기준일자     유형 연관검색어       검색일자         검색량
0   2024-03-07  일별급상승  소득공제 2021-06-30    0.000000
1   2024-03-07  일별급상승  소득공제 2021-07-28    0.000000
2   2024-03-07  일별급상승  소득공제 2021-08-25    0.000000
3   2024-03-07  일별급상승  소득공제 2021-09-22    0.000000
4   2024-03-07  일별급상승  소득공제 2021-10-20    0.000000
5   2024-03-07  일별급상승  소득공제 2021-11-17    0.000000
6   2024-03-07  일별급상승  소득공제 2021-12-15    0.000000
7   2024-03-07  일별급상승  소득공제 2022-01-12    0.000000
8   2024-03-07  일별급상승  소득공제 2022-02-09    0.000000
9   2024-03-07  일별급상승  소득공제 2022-03-09    0.000000
10  2024-03-07  일별급상승  소득공제 2022-04-06    0.000000
11  2024-03-07  일별급상승  소득공제 2022-05-04    0.000000
12  2024-03-07  일별급상승  소득공제 2022-06-01    0.000000
13  2024-03-07  일별급상승  소득공제 2022-06-29    0.000000
14  2024-03-07  일별급상승  소득공제 2022-07-27  

In [34]:
results[0][0]

,달러환율
date,
2020-03-06,22.93863
2020-03-07,10.50296
2020-03-08,8.70288
2020-03-09,32.45485
2020-03-10,32.28908
...,...
2024-03-02,12.69856
2024-03-03,11.73118
2024-03-04,25.15213


In [17]:
selected_keywords['연관검색어'].iloc[0]

'HBM관련주'

In [22]:
results[0][0].columns[0]
a=[]
for i in range(len(results)):
    for j in range(len(results[i])):
        a.append(results[i][j].columns[0])

In [23]:
a

['달러환율',
 '적금계산기',
 '다우지수',
 '적금',
 '코스닥',
 '증권',
 '증시',
 '파킹통장',
 '주식시세',
 '하이닉스주가',
 '저축은행예금',
 '기아차주가',
 '코스피지수',
 '주요증시',
 '예금이자계산기',
 'ISA계좌',
 '국내증시',
 '나스닥지수',
 '예금금리',
 '주가지수',
 'AI관련주',
 '대출',
 '저축은행적금',
 '삼성전기주가',
 '삼성전자주식',
 'CMA',
 '미국국채10년',
 '정기예금금리비교',
 'ETF',
 '한전주가',
 '금리',
 '엘지전자주가',
 '적금이자높은은행',
 '예금이자높은은행',
 '금리계산기',
 '하이브주가',
 '홍콩H',
 '도지코인시세',
 '미국국채수익률',
 '아모레퍼시픽주가',
 'IRP',
 '연금저축',
 '적금추천',
 '초전도체관련주',
 '적금이자계산기',
 '트레이딩뷰',
 '삼성전자우선주',
 '레고켐바이오주가',
 '나스닥선물지수',
 '국채금리',
 '나스닥',
 '유럽증시',
 'S&P500',
 '뉴욕증시',
 '미국주식',
 '종합주가지수',
 '코스피200',
 '신협정기예금금리',
 '삼성전자배당금',
 '예금이자',
 '채권',
 'IRP계좌개설',
 '퇴직금',
 '파킹통장금리비교',
 'CMA통장',
 '국내주식',
 '삼성전자주가',
 '에코프로주가',
 '공모주일정',
 'ELS',
 '한국비엔씨주가',
 '매도',
 '홍콩ELS',
 '반도체관련주',
 '2차전지관련주',
 'QQQ주가',
 '재테크',
 '소득공제',
 'CMA금리비교',
 'CD금리',
 '로봇관련주',
 '연말정산소득공제',
 '펀드',
 '환율전망',
 '금융',
 '정기적금금리비교',
 '경제뉴스',
 '채권투자방법',
 'KODEX레버리지',
 '수소관련주',
 '사모펀드',
 '방산관련주',
 '전환사채',
 '투자',
 '펀딩',
 '돈버는방법',
 '핀테크',
 '채권투자',
 '부동산PF',
 'IRP퇴직연금해지',
 'EL

In [24]:
df = pd.DataFrame(a, columns=['Column'])

# CSV 파일로 저장
file_path = 'example_list.csv'
df.to_csv(file_path, index=False)

file_path

'example_list.csv'